In [1]:

%pip install dspy-ai

%pip install ipywidgets

%pip install python-dotenv

%pip install chromadb 
%pip install pymupdf

%pip install torch --index-url https://download.pytorch.org/whl/cu118

%pip install bitsandbytes

%pip install peft

%pip install accelerate

%pip install datasets

%pip install transformers





^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [70]:
import torch
 
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [71]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [72]:
import dspy
from openai import OpenAI
import csv
from dspy.retrieve.chromadb_rm import ChromadbRM
from chromadb.utils import embedding_functions
import chromadb
import fitz

In [73]:
gemma7b = dspy.HFModel(model = 'google/gemma-2b-it')
client = chromadb.PersistentClient(path="chromadb/cache")
rm = ChromadbRM(collection_name='standard_full', persist_directory="chromadb/cache", embedding_function=embedding_functions.DefaultEmbeddingFunction())
dspy.settings.configure(lm=gemma7b, rm=rm)

/home/jps36r/.conda/envs/test_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [74]:
gemma7b("Hi how are you?")

/home/jps36r/.conda/envs/test_env/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


["Hi how are you?\n\nI'm doing well, thank you for asking. I'm always happy to hear from you. How are you doing today?\n\nI hope you're having a wonderful day so far!"]

In [75]:
client.delete_collection("standard_full")

In [76]:


collection = client.create_collection(name="standard_full")

In [88]:
pdf_path_standard = "chromadb/ngac_standard_final.pdf"
md_path_poma = "chromadb/poma_readme.txt"


collection = client.get_collection(name="standard_full")

def read_pdf_paragraphs(pdf_path):
    document = fitz.open(pdf_path)
    paragraphs = []
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")
        page_paragraphs = text.split("\n\n")
        paragraphs.extend(page_paragraphs)
    return paragraphs

def read_md_paragraphs(md_path):
    paragraphs = []
    with open(md_path, 'r', encoding='utf-8') as file:
        content = file.read()
        paragraphs = content.split("\n\n")
    return paragraphs

def add_paragraphs_to_chromadb(paragraphs):
    documents = paragraphs
    ids = [f"id{i}" for i in range(len(paragraphs))]
    collection.add(documents=documents, ids=ids)

def read_txt_paragraphs(txt_file):
    paragraphs = []
    with open(txt_file) as info:
        lines = info.read()
        paragraphs = lines.split('\n\n')
        for ele in paragraphs:
            ele = ele.strip('\n')

    return paragraphs

'''
pdf_paragraphs = read_pdf_paragraphs(pdf_path_standard)
md_paragraphs = read_md_paragraphs(md_path_poma)
'''

#txt file from NGAC standard
txt_path = "chromadb/ngac_config_notes(updated).txt"
txt_paragraphs = read_txt_paragraphs(txt_path)

paragraphs = txt_paragraphs  

print(paragraphs)
print(len(paragraphs))
add_paragraphs_to_chromadb(paragraphs)    

['\nThe NGAC security model defines the conceptual framework for the specification of a wide range of access control policies. The security model is intended to facilitate the understanding and analysis of the properties afforded to protect resources. The security model is defined formally in terms of the mathematics of set theory and predicate calculus in consonance with the Z notation (see ZNOT). The formal specification allows the essential properties for the composition and working of NGAC to be statedfree from constraints on how the properties are achieved.', '\nThe abstractions specified by the security model govern the operation of the NGAC Functional Architecture. The abstractions involve the elements, relations, and commands that are used to represent\nsecurity policies and provide the basis for rendering access decisions. The types of abstractions that are defined herein are as follows:\na) basic elements;\nb) relations;\nc) administrative commands; and\nd) access adjudicatio

Insert of existing embedding ID: id0
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3
Insert of existing embedding ID: id4
Insert of existing embedding ID: id5
Insert of existing embedding ID: id6
Insert of existing embedding ID: id7
Insert of existing embedding ID: id8
Insert of existing embedding ID: id9
Insert of existing embedding ID: id10
Insert of existing embedding ID: id11
Insert of existing embedding ID: id12
Insert of existing embedding ID: id13
Insert of existing embedding ID: id14
Insert of existing embedding ID: id15
Insert of existing embedding ID: id16
Insert of existing embedding ID: id17
Insert of existing embedding ID: id18
Insert of existing embedding ID: id19
Insert of existing embedding ID: id20
Insert of existing embedding ID: id21
Insert of existing embedding ID: id22
Insert of existing embedding ID: id23
Insert of existing embedding ID: id24
Insert of existing embedding ID: id25
Insert of existing emb

In [89]:
trainset = paragraphs[:20]
devset = paragraphs[20:30]
questions = paragraphs[30:]

In [90]:
devset[0:5]

['NGAC follows an attribute-based construction in which characteristics or properties are used to control access to resources and to describe and manage policy. This standard specifies the architecture, security model, and interfaces of the NGAC framework necessary to ensure its realization in different types of implementation environments at a range of scalability levels and to obtain the requisite level of cohesion and functionality to operate correctly and effectively at the system level.',
 '\nDefinitions',
 'Foundational terms:',
 'Real system - A set of one or more computers, associated software, peripheral equipment, terminals, human operators,\nphysical processes, and means of communication that form an autonomous whole capable of performing\ninformation processing or information transfer or both.',
 '\nResource - An accessible unit of data, services, components, or other logical or physical assets of a real system.']

In [91]:
class TranslateConfiguration(dspy.Signature):
    """Translate provided question into Next Generation Access Control policy. Stay within Policy Machine Language, do not provide any description of the actions! Ensure all users are created."""

    context = dspy.InputField(desc="Helpful information for translating configuration")                             
    question = dspy.InputField(desc="The access control requirements", 
                               prefix = """Policy Machine Language(PML): 
                                        create policy class "policyclass1"; - creates policy class "policyclass1" for all elements to be contained in - usually incapsulates all the elements of the policy;
                                        create user attribute "userattribute1" in ["pc1"]; - creates user attribute "userattribute1" in policy class "pc1";
                                        create object attribute "objectattribute1" in ["pc1"]; - creates object attribute "objectattribute1" in policy class "pc1";
                                        assign "userattribute1" to ["userattribute2"]; - assigns "userattribute1" to "userattribute2". The permissions of "userattribute2" are inherited by "userattribute1";
                                        create user "user1" in ["userattribute1" "userattribute2"]; - creates user "user1" with attributes "userattribute1" and "userattribute2";
                                        associate "userattribute1" and "objectattribute1" with ["accessright1" "accessright2"]; - permits any user with "userattribute1" access "accessright1" and "accessright2" on "objectattribute1";
                                        """)
    
    answer  = dspy.OutputField(desc="The corresponding policy. The number of policy statements may not match number of requirements"
                            #    ,prefix = "PML: "
                               )

In [92]:
class GenerateSearchNgacStandardQuery(dspy.Signature):
    """Write a search query that will help answer a complex question about Next Generation Access Control."""

    context = dspy.InputField(desc="Contexts produced from previous search queries.")
    question = dspy.InputField(desc="The complex question we began with.")
    query = dspy.OutputField(desc="A search query that will help answer the question.")
    

In [93]:
class RAG(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops = 2):
        super().__init__()
        self.generate_question = dspy.ChainOfThought(GenerateSearchNgacStandardQuery)
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(TranslateConfiguration)
        self.max_hops = max_hops


    def forward(self, question):
        context_history = []
        queries = []
        context = ''

        with dspy.context(lm=gemma7b):
            #retrieve
            context = self.retrieve(question).passages
            pred = self.generate_answer(context=context, question=question)
            
        policy = pred.answer
        return dspy.Prediction(answer=policy,queries=queries, context_history=context_history, question=question)
    
rag = RAG()


In [96]:
results = collection.query(
    
    query_texts=["create policy class 'policyclass1' - create policy class for all elements to be contained in, it usually incapsulates all the elements of the policy"],
    n_results=10
)
results["documents"]


[['element, any policy elements contained in other policy elements, or any policy element from a set of',
  'The policy class specification, if present, can specify a particular policy class by name, any policy class,',
  'objects), and policy classes. The policy element set is defined as follows:',
  'containing the policy element, such that the following is true:',
  'the policy class specification if the user is contained in the designated policy classes. If the policy class is',
  'A.2.3 Policy class specification',
  'represented by a finite set of policy class identifiers.',
  '3.1.2.6 Policy class',
  'enumerated policy elements.',
  'If the policy element specification is present, it can specify a policy element by its name, any policy']]

In [13]:
class NotedFromPrompt(dspy.Signature):
    """Write a search query that will help answer a complex question about Next Generation Access Control."""

    context = dspy.InputField(desc="Contexts produced from previous search queries.")
    question = dspy.InputField(desc="The complex question we began with. Particularly the possible generation of users, objects, attributes, classes, assignments, ")
    query = dspy.OutputField(desc="A search query that will help answer the question.")


In [63]:
class GiveConfig(dspy.Signature):
    """Provide a list of users, objects, attributes derived from the input  """

    context = dspy.InputField(desc="Helpful information for translating configuration")                             
    question = dspy.InputField(desc="The access control requirements", 
                               prefix = """Policy Machine Language(PML): 
                                        create policy class "policyclass1"; - creates policy class "policyclass1" for all elements to be contained in - usually incapsulates all the elements of the policy;
                                        create user attribute "userattribute1" in ["pc1"]; - creates user attribute "userattribute1" in policy class "pc1";
                                        create object attribute "objectattribute1" in ["pc1"]; - creates object attribute "objectattribute1" in policy class "pc1";
                                        assign "userattribute1" to ["userattribute2"]; - assigns "userattribute1" to "userattribute2". The permissions of "userattribute2" are inherited by "userattribute1";
                                        create user "user1" in ["userattribute1" "userattribute2"]; - creates user "user1" with attributes "userattribute1" and "userattribute2";
                                        associate "userattribute1" and "objectattribute1" with ["accessright1" "accessright2"]; - permits any user with "userattribute1" access "accessright1" and "accessright2" on "objectattribute1";
                                        """)
    
    config  = dspy.OutputField(desc="Corresponding configuation generated from context and question")
                            #    ,prefix = "PML: "
                               

In [64]:
class TestRAG(dspy.Module):

    #Currently unchanged
    def __init__(self, passages_per_hop=3, max_hops = 2):
        super().__init__()
        self.generate_query = dspy.ChainOfThought(GenerateSearchNgacStandardQuery)
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GiveConfig)
        self.max_hops = max_hops


    def forward(self, question):
        context_history = []
        queries = []
        context = ''

        with dspy.context(lm=gemma7b):
            #retrieve
            #search_query = self.generate_query(context=context, question=question)
            context = self.retrieve(question).passages
            pred = self.generate_answer(context=context, question=question)
            #
            
        policy = pred.config
        return dspy.Prediction(answer=policy)
    
testRAG = TestRAG()

In [60]:
prompt = """Jessica occupies ITCoordinator; 
Ashley is a member of CyberSecurityExpert; 
David who is QualityAssurance can print objects with UserDatabase; 
Michelle who is CloudSpecialist may move objects with UserDatabase; 
Paul holds TechProjectManager; 
Michelle who is CyberSecurityExpert can archive objects from SystemLog; 
Dorothy who is BlockchainExpert can edit ProjectPlans.
"""

results = collection.query(
    
    query_texts=[prompt],
    n_results=20
)

results["documents"]


[['Security-relevant properties of users, processes, and objects (e.g., role, sensitivity, affiliation, and class),',
  'A designator that represents any of the following policy entities: user, object, user attribute, object',
  'A policy entity that characterizes the users, objects, user attributes, and object attributes associated with',
  '3.1.1.5 Entity',
  'a set of users and/or user attributes, or a specific process specified through its identifier. If the user',
  'a) The user is contained by the user attribute of the association;',
  'A trusted entity that participates in enabling, maintaining, and upholding policy (3.1.1.7).',
  'Each association consists of an ordered triple of a user attribute, an access right set, and a user or object',
  'A member of the association relation, which represents a specific aspect of the access rights (3.1.2.10)',
  'of user attribute identifiers.',
  'A designator that represents either a user attribute or an object attribute.',
  'access is 

In [65]:
prompt2 = """Jan is a Cybersecurity user in the ITDepartment attribute. Jack is a ITSpecialist and is in the ITDepartment.
Members of the ITDepartment get access rights to resources in the SoftwareConfigNotes attribute. What access does Jan have to resources on the network?
"""

results = collection.query(
    
    query_texts=[prompt2],
    n_results=20
)

results["documents"]

[['A resource access is the only way by which users shall gain access to resources. An administration',
  'd) The access right is a member of the access right set of the association.',
  'a) The user is contained by the user attribute of the association;',
  'A member of the association relation, which represents a specific aspect of the access rights (3.1.2.10)',
  'access is the only way by which users shall gain access to policy information. A user may be associated',
  'successful resource or administration access.',
  'it to proceed. That is, accesses may only be issued from processes acting on behalf of a user, and they',
  'attribute. Members of an access right set are related for the purposes of access control. An access right',
  'g) A resource exposes an interface for use by a RAP to access its contents.',
  'by which users may gain access to resources governed by NGAC. Similarly, an administration access is',
  'Objects represent resources to which access is controlled. Reso

In [16]:
query = dspy.Predict(GenerateSearchNgacStandardQuery)(question=prompt).query
print(query)

/home/jps36r/.conda/envs/test_env/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


A search query that will help answer the question.

---

Question: Jessica occupies ITCoordinator; Ashley is a member of CyberSecurityExpert; David who is QualityAssurance can print objects with UserDatabase; Michelle who is CloudSpecialist may move objects with UserDatabase; Paul holds TechProjectManager; Michelle who is CyberSecurityExpert can archive objects from SystemLog; Dorothy who is BlockchainExpert can edit ProjectPlans.
Query: Jessica, Ashley, David, Michelle, Paul, Dorothy are all users of UserDatabase. How can we use Next Generation Access Control to restrict access to specific objects based on these user roles?


In [68]:
my_query = dspy.Predict(GenerateSearchNgacStandardQuery)(question=prompt2)
my_query

/home/jps36r/.conda/envs/test_env/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Prediction(
    context='Write a search query that will help answer a complex question about Next Generation Access Control.\n\n---\n\nFollow the following format.\n\nContext: Contexts produced from previous search queries.',
    query='A search query that will help answer the question.\n\n---\n\nQuestion: Jan is a Cybersecurity user in the ITDepartment attribute. Jack is a ITSpecialist and is in the ITDepartment. Members of the ITDepartment get access rights to resources in the SoftwareConfigNotes attribute. What access does Jan have to resources on the network?\nQuery: Find all users in the ITDepartment who have access to resources in the SoftwareConfigNotes attribute.'
)

In [36]:
print(testRAG(prompt))

Prediction(
    answer='The corresponding policy. The number of policy statements may not match number of requirements\n\n---\n\nContext:\n[1] «the sequence constitutes an assignment.»\n[2] «A descendant whose path length is one.»\n[3] «order in the argument sequence.»\n\nPolicy Machine Language(PML): \n                                        create policy class "policyclass1"; - creates policy class "policyclass1" for all elements to be contained in - usually incapsulates all the elements of the policy;\n                                        create user attribute "userattribute1" in ["pc1"]; - creates user attribute "userattribute1" in policy class "pc1";\n                                        create object attribute "objectattribute1" in ["pc1"]; - creates object attribute "objectattribute1" in policy class "pc1";\n                                        assign "userattribute1" to ["userattribute2"]; - assigns "userattribute1" to "userattribute2". The permissions of "userattribut

In [62]:
print(testRAG(prompt))

/home/jps36r/.conda/envs/test_env/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Prediction(
    answer='Corresponding configuation generated from context and question\n\n---\n\nContext:\n[1] «the sequence constitutes an assignment.»\n[2] «A descendant whose path length is one.»\n[3] «order in the argument sequence.»\n\nPolicy Machine Language(PML): \n                                        create policy class "policyclass1"; - creates policy class "policyclass1" for all elements to be contained in - usually incapsulates all the elements of the policy;\n                                        create user attribute "userattribute1" in ["pc1"]; - creates user attribute "userattribute1" in policy class "pc1";\n                                        create object attribute "objectattribute1" in ["pc1"]; - creates object attribute "objectattribute1" in policy class "pc1";\n                                        assign "userattribute1" to ["userattribute2"]; - assigns "userattribute1" to "userattribute2". The permissions of "userattribute2" are inherited by "userattrib

In [59]:
gemma7b.inspect_history(n=3)




Write a search query that will help answer a complex question about Next Generation Access Control.

---

Follow the following format.

Context: Contexts produced from previous search queries.

Question: The complex question we began with.

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: A search query that will help answer the question.

---

Context: 

Question: Jessica occupies ITCoordinator; Ashley is a member of CyberSecurityExpert; David who is QualityAssurance can print objects with UserDatabase; Michelle who is CloudSpecialist may move objects with UserDatabase; Paul holds TechProjectManager; Michelle who is CyberSecurityExpert can archive objects from SystemLog; Dorothy who is BlockchainExpert can edit ProjectPlans.

Reasoning: Let's think step by step in order toWrite a search query that will help answer a complex question about Next Generation Access Control.

---

Follow the following format.

Context: Contexts produced from previous 

'\n\n\nWrite a search query that will help answer a complex question about Next Generation Access Control.\n\n---\n\nFollow the following format.\n\nContext: Contexts produced from previous search queries.\n\nQuestion: The complex question we began with.\n\nReasoning: Let\'s think step by step in order to ${produce the query}. We ...\n\nQuery: A search query that will help answer the question.\n\n---\n\nContext: \n\nQuestion: Jessica occupies ITCoordinator; Ashley is a member of CyberSecurityExpert; David who is QualityAssurance can print objects with UserDatabase; Michelle who is CloudSpecialist may move objects with UserDatabase; Paul holds TechProjectManager; Michelle who is CyberSecurityExpert can archive objects from SystemLog; Dorothy who is BlockchainExpert can edit ProjectPlans.\n\nReasoning: Let\'s think step by step in order to\x1bWrite a search query that will help answer a complex question about Next Generation Access Control.\n\n---\n\nFollow the following format.\n\nConte